# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Подключим библиотеки, необходимые для проведения обучения и тестирования модели.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Загружаем файл с данными

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
    print('Файл загружен')
except:
    print('Файл не найден')

Файл загружен


Проверим датафрейм на пропуски и ошибки при загрузке данных.

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разделим датафрейм на два, с признаками(features) и целевым столюцом(target).

Общую выборку перед обучением модели требуется разбить на 3 части: обучающую, валидационную и тестовую выборки. Разбиваем на выборки в соотношении 60%/20%20%

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

Проверим размеры выборок.

In [6]:
print('Размер обучающей выборки:', target_train.shape[0], 'Составляет', round(target_train.shape[0]/df.shape[0]*100, 2), '% от общей')
print('Размер валидационной выборки:', target_valid.shape[0], 'Составляет', round(target_valid.shape[0]/df.shape[0]*100, 2), '% от общей')
print('Размер тестовой выборки:', target_test.shape[0], 'Составляет', round(target_test.shape[0]/df.shape[0]*100, 2), '% от общей')

Размер обучающей выборки: 1928 Составляет 59.99 % от общей
Размер валидационной выборки: 643 Составляет 20.01 % от общей
Размер тестовой выборки: 643 Составляет 20.01 % от общей


Выведем первые 5 строк целевого столбца тестовой и валидационной выборки, что бы убедиться что они не пересекаются.

In [7]:
display(target_valid.head())
display(target_test.head())

1386    0
3124    0
1956    0
2286    0
3077    0
Name: is_ultra, dtype: int64

160     0
2498    0
1748    0
1816    0
1077    1
Name: is_ultra, dtype: int64

## Исследуйте модели

В текущем исследовании будем рассматрицать следующий модели машинного обучения: Дерево Решений(DecisionTreeClassifier), Случайный Лес(RandomForestClassifier) и Логистическая регрессия(LogisticRegression).

### DecisionTreeClassifier

Обучим модель DecisionTreeClassifier с гиперпараметрами по умолчанию. Начальное состояние генератора случайных чисел зададим 12345.

In [8]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
current_score = accuracy_score(target_valid, predictions)
print('Точность обучения:', current_score)

Точность обучения: 0.713841368584759


Попробуем увеличить точность обучения изменяя гиперпараметры модели. Подберем максимальное количество фичей max_features, которые рассматриваются при выборе лучшего разбиения. 

Лучший гиперпараметр сохраняем в соответствующей переменной, что бы была возможность использовать его в дальнейшем. 

Модель обучения с лучшими показателями сохраняем в переменную tree_model.

In [9]:
max_score = 0
for max_features in ['auto', 'sqrt', 'log2']:
    model = DecisionTreeClassifier(random_state=12345, max_features=max_features)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score < current_score:
        best_max_features = max_features
        max_score = current_score
        tree_model = model
print('Максимальное количество фичей:', best_max_features)
print('Точность обучения:', max_score)

Максимальное количество фичей: auto
Точность обучения: 0.7231726283048211


Подберем гиперпараметр глубины дерева.

In [10]:
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, max_features=best_max_features)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score < current_score:
        max_depth = depth
        max_score = current_score
        tree_model = model
print('Максимальная глубина:', max_depth)
print('Точность обучения:', max_score)

Максимальная глубина: 5
Точность обучения: 0.7884914463452566


Подберем гиперпараметр миниального количества экземпляров, которое может содержаться в узле для его дальнейшего разбиения.

In [11]:
for min_samples_split in range(2, 100):
    model = DecisionTreeClassifier(random_state=12345, max_depth=max_depth, max_features=best_max_features, min_samples_split=min_samples_split)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score <= current_score:
        best_min_samples_split = min_samples_split
        max_score = current_score
        tree_model = model
print('Миниального количества экземпляров в узле :', best_min_samples_split)
print('Точность обучения:', max_score)

Миниального количества экземпляров в узле : 23
Точность обучения: 0.7884914463452566


Подберем миниальное количество экземпляров, которое может содержаться в листе.

In [12]:
for min_samples_leaf in range(1, 100):
    model = DecisionTreeClassifier(random_state=12345, max_depth=max_depth, max_features=best_max_features, 
                                   min_samples_split=best_min_samples_split, min_samples_leaf=min_samples_leaf)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score <= current_score:
        best_min_samples_leaf = min_samples_leaf
        max_score = current_score
        tree_model = model
print('миниальное количество экземпляров в листе:', best_min_samples_leaf)
print('Точность обучения:', max_score)

миниальное количество экземпляров в листе: 1
Точность обучения: 0.7884914463452566


Точность модели удалось поднять за счет гиперпараметров глубины и максимального количества фичей. 

min_samples_split и min_samples_leaf не повлияли на точность модели.

### RandomForestClassifier

Обучим модель RandomForestClassifier с гиперпараметрами по умолчанию. Начальное состояние генератора случайных чисел зададим 12345.

In [13]:
model = RandomForestClassifier(random_state=12345) 
model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
current_score = accuracy_score(target_valid, predictions)
print('Точность обучения:', current_score)

Точность обучения: 0.7853810264385692


Попробуем увеличить точность обучения изменяя гиперпараметры модели. Подберем колличество деревеьев в модели

Лучший гиперпараметр сохраняем в соответствующей переменной, что бы была возможность использовать его в дальнейшем.

Модель обучения с лучшими показателями сохраняем в переменную forest_model.

In [14]:
max_score = 0
for n_estimators in range(10, 200, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=n_estimators) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score < current_score:
        best_n_estimators = n_estimators 
        max_score = current_score
        forest_model = model
print('Колличество деревьев:', best_n_estimators)
print('Точность обучения:', max_score)

Колличество деревьев: 50
Точность обучения: 0.7916018662519441


Подберем максимальную глубину дерева.

In [15]:
for max_depth in range(2, 50):
    model = RandomForestClassifier(random_state=12345, n_estimators=best_n_estimators, max_depth=max_depth) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score < current_score:
        best_max_depth = max_depth 
        max_score = current_score
        forest_model = model
print('Максимальная глубина:', best_max_depth)
print('Точность обучения:', max_score)

Максимальная глубина: 8
Точность обучения: 0.807153965785381


Подберем гиперпараметр миниального количества экземпляров, которое может содержаться в узле для его дальнейшего разбиения.

In [16]:
for min_samples_split in range(2, 50):
    model = RandomForestClassifier(random_state=12345, n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=min_samples_split) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions) 
    if max_score < current_score:
        best_min_samples_split = min_samples_split 
        max_score = current_score
        forest_model = model
print('миниальное количество экземпляров в узле:', best_min_samples_split)
print('Точность обучения:', max_score)

миниальное количество экземпляров в узле: 18
Точность обучения: 0.8102643856920684


Подберем миниальное количество экземпляров, которое может содержаться в листе.

In [17]:
for min_samples_leaf in range(1, 50):
    model = RandomForestClassifier(
        random_state=12345, n_estimators=50, max_depth=8, min_samples_split=best_min_samples_split, min_samples_leaf=min_samples_leaf
    ) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions) 
    if max_score < current_score:
        best_min_samples_leaf = min_samples_leaf 
        max_score = current_score
        forest_model = model
print('Миниальное количество экземпляров в листе:', best_min_samples_leaf)
print('Точность обучения:', max_score)

Миниальное количество экземпляров в листе: 1
Точность обучения: 0.8102643856920684


Модель случайного леса хорошо обучается, и подбор каждого из выбранных гиперпараметров улучшает качество модели.

### LogisticRegression

Обучим модель LogisticRegression с гиперпараметрами по умолчанию. Начальное состояние генератора случайных чисел зададим 12345.

In [18]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
current_score = accuracy_score(target_valid, predictions)
print('Точность обучения:', current_score)

Точность обучения: 0.7107309486780715


Попробуем увеличить точность обучения изменяя гиперпараметры модели. Подберем алгоритм для использования в обучении.

Лучший гиперпараметр сохраняем в соответствующей переменной, что бы была возможность использовать его в дальнейшем.

Модель обучения с лучшими показателями сохраняем в переменную logistic_model.

In [19]:
max_score = 0
for solver in ['lbfgs', 'liblinear', 'sag', 'saga']:
    model = LogisticRegression(random_state=12345, solver=solver, max_iter=10000)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score < current_score:
        best_solver = solver
        max_score = current_score
        logistic_model = model
print('Алгоритм для использования в обучении:', best_solver)
print('Точность обучения:', max_score)

Алгоритм для использования в обучении: lbfgs
Точность обучения: 0.7107309486780715


Подберем максимальное коллчисество итераций.

In [20]:
for max_iter in range(100, 10000, 100):
    model = LogisticRegression(random_state=12345, solver=best_solver, max_iter=max_iter)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score <= current_score:
        best_max_iter = max_iter
        max_score = current_score
        logistic_model = model
print('Размер максимального коллчисество итераций:', best_max_iter)
print('Точность обучения:', max_score)

Размер максимального коллчисество итераций: 9900
Точность обучения: 0.7107309486780715


Подберем штраф за неверную итерацию

In [21]:
for C in np.logspace(-4, 4, 10):
    model = LogisticRegression(random_state=12345, solver=best_solver, max_iter=best_max_iter, C=C)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score <= current_score:
        regularization_strength = C
        max_score = current_score
        logistic_model = model
print('Размер штрафа неверной итерации:', regularization_strength)
print('Точность обучения:', max_score)

Размер штрафа неверной итерации: 10000.0
Точность обучения: 0.7558320373250389


Подберем толерантность к критериям остановки.

In [22]:
for tol in np.logspace(-4, 4, 100):
    model = LogisticRegression(random_state=12345, solver=best_solver, C=regularization_strength, tol=tol)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    current_score = accuracy_score(target_valid, predictions)
    if max_score <= current_score:
        best_tol = tol
        max_score = current_score
        logistic_model = model
print('Толерантность к критериям остановки:', best_tol)
print('Точность обучения:', max_score)

Толерантность к критериям остановки: 114.9756995397738
Точность обучения: 0.7558320373250389


Из выбранных гиперпараметров увеличение точности дало только изменение штрафа неверной итерации.

## Проверьте модель на тестовой выборке

Проверим лучшие модели обучения на тестовой выборке

In [23]:
tree_model.fit(features_train, target_train)
predictions = tree_model.predict(features_test)
print('Точность обучения на тестовой выборке:', accuracy_score(target_test, predictions))
predictions = tree_model.predict(features_valid)
print('Точность обучения на валидационной выборке:', accuracy_score(target_valid, predictions))

Точность обучения на тестовой выборке: 0.7884914463452566
Точность обучения на валидационной выборке: 0.7884914463452566


In [24]:
forest_model.fit(features_train, target_train)
predictions = forest_model.predict(features_test)
print('Точность обучения на тестовой выборке:', accuracy_score(target_test, predictions))
predictions = forest_model.predict(features_valid)
print('Точность обучения на валидационной выборке:', accuracy_score(target_valid, predictions))

Точность обучения на тестовой выборке: 0.807153965785381
Точность обучения на валидационной выборке: 0.8102643856920684


In [25]:
logistic_model.fit(features_train, target_train)
predictions = logistic_model.predict(features_test)
print('Точность обучения на тестовой выборке:', accuracy_score(target_test, predictions))
predictions = logistic_model.predict(features_valid)
print('Точность обучения на валидационной выборке:', accuracy_score(target_valid, predictions))

Точность обучения на тестовой выборке: 0.7387247278382582
Точность обучения на валидационной выборке: 0.7558320373250389


Модель Случайного Леса на тестовой выборке так же показывает высокую точность обучения как и на валидационной.

Модель Дерева Решений показывает одинаковую точность как на тестовой выборке, так и на валидационной и занимает второе место.

Модель Логистической Регрессии показывает самые худшие показатели точности и на тестовой выборке не проходит порог 0.75

## (бонус) Проверьте модели на адекватность

Проверим модель на адекватность с помощью DummyClassifier. DummyClassifier всегда предсказывает наиболее часто встречающийся класс.

In [26]:
dummy_model = DummyClassifier(random_state=12345)
dummy_model.fit(features_train, target_train)
predictions = dummy_model.predict(features_test)
print('Точность обучения:', accuracy_score(target_test, predictions))

Точность обучения: 0.6842923794712286
